In [10]:
import pandas as pd
import folium
from math import sqrt

In [75]:
def colors(conc_nrm):
    """Receives a normalized concentration value and returns a hex depending on the air quality level.

    Args:
        conc_nrm (float): result of the division between the averaged concentration and a bad quality concentration.

    Returns:
        str : hex color
    """

    if conc_nrm<=0.25:
        color = '#3CB371'
    elif conc_nrm<=0.5:
        color = '#FFD700'
    elif conc_nrm<=0.75:
        color = '#FF7F50'
    else:
        color = '#483D8B'
        
    return color

def interpolate_aqdata_date(city, pollutant, date):
    """ Creates a folium map with stations arranged by size and colour depending on the concentration of the pollutant.

    Args:
        city (str): code for the city to be analysed, for example: cdmx
        pollutant (str): pollutant to be plotted
        date (str): date to be analysed

    Returns:
        folium map
    """
    dir_pcs = '../data/processed/'

    city_stations = pd.read_csv('../data/raw/Grl/stations/city_stations.csv')

    city_dict = {'cdmx':'Valle de México'}
    
    p_limits = {'PM10':214, 'O3':154, 'CO':16.5,
               'PM25':97.4, 'SO2':195,'NO2':315}
    
    data_bydateParam = pd.read_csv(dir_pcs+city+'/'+city+'_2017-2020_filtered_'+pollutant+'.csv').set_index('FECHA')
    
    #lists to append valid values of lat and long for interpolation
    
    x = []
    y = []
    
    for i, est in city_stations[city_stations['city']==city_dict[city]].iterrows():

                est_code = city_stations.loc[(i),'codigo']
                c_value = data_bydateParam.loc[(date),est_code]
                
                if pd.notna(c_value):
                    x.append(est.long)
                    y.append(est.lat)

    #Registers the boundries coordinates for the interpolation
    min_x = min(x)
    max_x = max(x)
    min_y = min(y)
    max_y = max(y)
    
    cnt_x = (min_x+max_x)/2
    cnt_y = (min_y+max_y)/2
    
    
    centro_lat, centro_lon = cnt_y, cnt_x #Centro del mapa

    #Creacion del mapa
    folium_map = folium.Map(location=[centro_lat,centro_lon], zoom_start=10,
                            tiles = 'cartodb positron')

    for i, est in city_stations[city_stations['city']==city_dict[city]].iterrows():

        est_code = city_stations.loc[(i),'codigo']

        #Coloca los marcadores en el mapa
        c_value = data_bydateParam.loc[(date),est_code]
        c_graph = (data_bydateParam.loc[(date),est_code])/p_limits[pollutant]

        #Puntos con nombre, latitud y longitud
        popup_text = f"<b> Nombre: </b> {est.nombre} <br> <b> Latitud: </b> {est.lat:.5f} <br> <b> Longitud: </b> {est.long:.5f} <br> <b> Contaminante: </b> {pollutant} <br> <b> Conc: </b> {c_value} <br>"

        #Coloca los marcadores en el mapa
        folium.CircleMarker(location=[est.lat, est.long], radius=c_graph*50,
                            tooltip = popup_text, fill=True, color=colors(c_graph),
                            fill_opacity=0.65).add_to(folium_map)
        
    #Valor de celda
    cellsize = 0.01
    #Valor de potencia
    p = 2
    
    #x and y values for the start of the interpolation
    xidw=min_x
    yidw=min_y

    #Variables for interpolation
    dividendo = 0
    divisor = 0
    idw=[]
    cont=0

    while xidw <= max_x:
        while yidw <= max_y:
            for i, est in city_stations[city_stations['city']==city_dict[city]].iterrows():

                est_code = city_stations.loc[(i),'codigo']
                c_value = data_bydateParam.loc[(date),est_code]
                
                if pd.notna(c_value):
                    dividendo = (c_value/(sqrt((est.long-xidw)**2+(est.lat-yidw)**2)**(p)))+ dividendo
                    divisor = (1/(sqrt((est.long-xidw)**2+(est.lat-yidw)**2)**(p))) + divisor
                    
            concentracion = dividendo/divisor
            #Aqui se guardan los valores de las concentraciones para usarlos despues
            
            c_graph = concentracion/p_limits[pollutant]
            
            #Puntos con nombre, latitud y longitud
            popup_text = f"<b> Nombre: </b> {'NA'} <br> <b> Latitud: </b> {yidw:.5f} <br> <b> Longitud: </b> {xidw:.5f} <br> <b> Contaminante: </b> {pollutant} <br> <b> Conc: </b> {concentracion} <br>"

            #Coloca los marcadores en el mapa
            folium.CircleMarker(location=[yidw, xidw], radius=1,
                                tooltip = popup_text, fill=True, color=colors(c_graph),
                                opacity=0.45).add_to(folium_map)
            
            idw.append([yidw,xidw,concentracion])
            yidw = yidw + cellsize
            dividendo = 0
            divisor = 0  
        xidw = xidw + cellsize
        yidw = min_y

    inter = pd.DataFrame(idw, columns=['lat','long','conc'])
    

    return(folium_map)

In [76]:
interpolate_aqdata_date('cdmx','O3','2020-03-31')

In [2]:
dir_pcs = '../data/processed/'
city_stations = pd.read_csv('../data/raw/Grl/stations/city_stations.csv')

In [6]:
city_stations[city_stations['city']=='Valle de México'].lat.min()

19.154286111111

In [7]:
inter = pd.DataFrame(columns=['lat','long','conc'])
inter

,lat,long,conc


In [40]:
def interpolation_tmp(city, pollutant, date):
    
    dir_pcs = '../data/processed/'

    city_stations = pd.read_csv('../data/raw/Grl/stations/city_stations.csv')

    city_dict = {'cdmx':'Valle de México'}
    
    p_limits = {'PM10':214, 'O3':154, 'CO':16.5,
               'PM25':97.4, 'SO2':195,'NO2':315}
    
    data_bydateParam = pd.read_csv(dir_pcs+city+'/'+city+'_2017-2020_filtered_'+pollutant+'.csv').set_index('FECHA')
    
    centro_lat, centro_lon = 19.442810, -99.131233 #Centro del mapa

    #Creacion del mapa
    folium_map = folium.Map(location=[centro_lat,centro_lon], zoom_start=10,
                            tiles = 'cartodb positron')
    
    
    #Valor de celda
    cellsize = 0.0042
    #Valor de potencia
    p = 2

    #Registers the boundries coordinates for the interpolation
    min_x=city_stations[city_stations['city']==city_dict[city]].long.min()
    max_x=city_stations[city_stations['city']==city_dict[city]].long.max()
    min_y=city_stations[city_stations['city']==city_dict[city]].lat.min()
    max_y=city_stations[city_stations['city']==city_dict[city]].lat.max()

    #x and y values for the start of the interpolation
    xidw=min_x
    yidw=min_y

    #Variables for interpolation
    dividendo = 0
    divisor = 0
    idw=[]
    cont=0

    while xidw <= max_x:
        while yidw <= max_y:
            for i, est in city_stations[city_stations['city']==city_dict[city]].iterrows():

                est_code = city_stations.loc[(i),'codigo']
                c_value = data_bydateParam.loc[(date),est_code]
                
                if pd.notna(c_value):
                    dividendo = (c_value/(sqrt((est.long-xidw)**2+(est.lat-yidw)**2)**(p)))+ dividendo
                    divisor = (1/(sqrt((est.long-xidw)**2+(est.lat-yidw)**2)**(p))) + divisor
                    
            concentracion = dividendo/divisor
            #Aqui se guardan los valores de las concentraciones para usarlos despues
            
            c_graph = concentracion/p_limits[pollutant]
            
            #Puntos con nombre, latitud y longitud
            popup_text = f"<b> Nombre: </b> {'NA'} <br> <b> Latitud: </b> {yidw:.5f} <br> <b> Longitud: </b> {xidw:.5f} <br> <b> Contaminante: </b> {pollutant} <br> <b> Conc: </b> {concentracion} <br>"

            #Coloca los marcadores en el mapa
            folium.CircleMarker(location=[yidw, xidw], radius=1,
                                tooltip = popup_text, fill=True, color=colors(c_graph),
                                fill_opacity=0.75).add_to(folium_map)
            
            idw.append([yidw,xidw,concentracion])
            yidw = yidw + cellsize
            dividendo = 0
            divisor = 0  
        xidw = xidw + cellsize
        yidw = min_y

    inter = pd.DataFrame(idw, columns=['lat','long','conc'])
    
    return (folium_map)



In [41]:
interpolation_tmp('cdmx','PM10','2020-03-31')

KeyboardInterrupt: 

In [69]:
def colors(conc_nrm):
    """Receives a normalized concentration value and returns a hex depending on the air quality level.

    Args:
        conc_nrm (float): result of the division between the averaged concentration and a bad quality concentration.

    Returns:
        str : hex color
    """

    if conc_nrm<=0.25:
        color = '#3CB371'
    elif conc_nrm<=0.5:
        color = '#FFD700'
    elif conc_nrm<=0.75:
        color = '#FF7F50'
    else:
        color = '#483D8B'
        
    return color

def visualize_aqdata_date(city, pollutant, date):
    """ Creates a folium map with stations arranged by size and colour depending on the concentration of the pollutant.

    Args:
        city (str): code for the city to be analysed, for example: cdmx
        pollutant (str): pollutant to be plotted
        date (str): date to be analysed

    Returns:
        folium map
    """
    dir_pcs = '../data/processed/'

    city_stations = pd.read_csv('../data/raw/Grl/stations/city_stations.csv')

    city_dict = {'cdmx':'Valle de México'}
    
    p_limits = {'PM10':214, 'O3':154, 'CO':16.5,
               'PM25':97.4, 'SO2':195,'NO2':315}
    
    data_bydateParam = pd.read_csv(dir_pcs+city+'/'+city+'_2017-2020_filtered_'+pollutant+'.csv').set_index('FECHA')
    
    #lists to append valid values of lat and long for interpolation
    
    x = []
    y = []
    
    for i, est in city_stations[city_stations['city']==city_dict[city]].iterrows():

                est_code = city_stations.loc[(i),'codigo']
                c_value = data_bydateParam.loc[(date),est_code]
                
                if pd.notna(c_value):
                    x.append(est.long)
                    y.append(est.lat)

    #Registers the boundries coordinates for the interpolation
    min_x = min(x)
    max_x = max(x)
    min_y = min(y)
    max_y = max(y)
    
    cnt_x = (min_x+max_x)/2
    cnt_y = (min_y+max_y)/2
    
    
    centro_lat, centro_lon = cnt_y, cnt_x #Centro del mapa

    #Creacion del mapa
    folium_map = folium.Map(location=[centro_lat,centro_lon], zoom_start=10,
                            tiles = 'cartodb positron')

    for i, est in city_stations[city_stations['city']==city_dict[city]].iterrows():

        est_code = city_stations.loc[(i),'codigo']

        #Coloca los marcadores en el mapa
        c_value = data_bydateParam.loc[(date),est_code]
        c_graph = (data_bydateParam.loc[(date),est_code])/p_limits[pollutant]

        #Puntos con nombre, latitud y longitud
        popup_text = f"<b> Nombre: </b> {est.nombre} <br> <b> Latitud: </b> {est.lat:.5f} <br> <b> Longitud: </b> {est.long:.5f} <br> <b> Contaminante: </b> {pollutant} <br> <b> Conc: </b> {c_value} <br>"

        #Coloca los marcadores en el mapa
        folium.CircleMarker(location=[est.lat, est.long], radius=c_graph*50,
                            tooltip = popup_text, fill=True, color=colors(c_graph),
                            fill_opacity=0.65).add_to(folium_map)

    return(folium_map)

def interpolation_tmp2(city, pollutant, date):
    
    dir_pcs = '../data/processed/'

    city_stations = pd.read_csv('../data/raw/Grl/stations/city_stations.csv')

    city_dict = {'cdmx':'Valle de México'}

    p_limits = {'PM10':214, 'O3':154, 'CO':16.5,
               'PM25':97.4, 'SO2':195,'NO2':315}

    data_bydateParam = pd.read_csv(dir_pcs+city+'/'+city+'_2017-2020_filtered_'+pollutant+'.csv').set_index('FECHA')
    
    #lists to append valid values of lat and long for interpolation
    
    x = []
    y = []
    
    for i, est in city_stations[city_stations['city']==city_dict[city]].iterrows():

                est_code = city_stations.loc[(i),'codigo']
                c_value = data_bydateParam.loc[(date),est_code]
                
                if pd.notna(c_value):
                    x.append(est.long)
                    y.append(est.lat)

    #Registers the boundries coordinates for the interpolation
    min_x = min(x)
    max_x = max(x)
    min_y = min(y)
    max_y = max(y)

    visualize_aqdata_date(city, pollutant, date)

    #Valor de celda
    cellsize = 0.01
    #Valor de potencia
    p = 2

    #x and y values for the start of the interpolation
    xidw=min_x
    yidw=min_y

    #Variables for interpolation
    dividendo = 0
    divisor = 0
    idw=[]
    cont=0

    while xidw <= max_x:
        while yidw <= max_y:
            for i, est in city_stations[city_stations['city']==city_dict[city]].iterrows():

                est_code = city_stations.loc[(i),'codigo']
                c_value = data_bydateParam.loc[(date),est_code]

                if pd.notna(c_value):
                    dividendo = (c_value/(sqrt((est.long-xidw)**2+(est.lat-yidw)**2)**(p)))+ dividendo
                    divisor = (1/(sqrt((est.long-xidw)**2+(est.lat-yidw)**2)**(p))) + divisor

            concentracion = dividendo/divisor
            #Aqui se guardan los valores de las concentraciones para usarlos despues

            c_graph = concentracion/p_limits[pollutant]

            #Puntos con nombre, latitud y longitud
            popup_text = f"<b> Nombre: </b> {'NA'} <br> <b> Latitud: </b> {yidw:.5f} <br> <b> Longitud: </b> {xidw:.5f} <br> <b> Contaminante: </b> {pollutant} <br> <b> Conc: </b> {concentracion} <br>"

            #Coloca los marcadores en el mapa
            folium.CircleMarker(location=[yidw, xidw], radius=1,
                                tooltip = popup_text, fill=True, color=colors(c_graph),
                                opacity=0.45).add_to(folium_map)

            idw.append([yidw,xidw,concentracion])
            yidw = yidw + cellsize
            dividendo = 0
            divisor = 0  
        xidw = xidw + cellsize
        yidw = min_y

    inter = pd.DataFrame(idw, columns=['lat','long','conc'])


    return(folium_map)
    

In [70]:
interpolation_tmp2('cdmx','PM10','2020-03-31')

NameError: name 'folium_map' is not defined